# NOTEBOOK PARA OBTENER LOS HITS Y MEJORES HITS
Esto incluye la limpieza que se requiere ya que se tienen dobles espacios en cada línea.

**Que pasos debo seguir para elegir todos los raw hits de acuerdo a lo indicado por Salva:**

"...En realidad, hablando con mi hermano hemos decidido que los selected pulses no cal guardarlos, los usas para el match, y coges todos los hits que coincidan (cada uno de ellos será una nueva fila en tu CVS). Como decía antes, en las columnas tendrás toda la información del muón, del bbfit, del affit, y del hit coincidente con un selected pulse particular."

El pseudocódigo que se me ocurre es el siguiente:

1. Seleccionar el primer "selected pulse" de la lista de seleccionados.
2. Filtrar el df de raw hits por todos los que coincidan en linea detector y piso detector.
3. Filtrar el resultado viendo cuales coinciden en los parámetrox X', Y', Z' y X, Y, Z.
4. Guardar el resultado en el dataframe final junto con runID, frameID
5. Iterar hasta que ya no haya mas selected pulses
6. Hacer lo mismo para cada evento del archivo seleccionado

Actualización: Salva me mencionó en correo del día 3 Jul que se debe filtrar con respecto al piso_detector y línea_detector y quedarse con todos esos hits. Cito a la letra: 

"Perdón por el mareo. A ver, hablando con mi hermano me ha recordado que la forma que quedamos de hacerlo es con match de linea y piso del sensor, mas un cutoff de tiempo relativo respecto al selected pulse, que dijimos por ejemplo 50.
Yo recomendaria este filtro ponerlo mas adelante y coger todos los raw hits qcoinciden en linia y piso, mas qnada porque ai queremos probar otro cutoff de 70 o 100, no tengamos que correr todo esto de nuevo..."

Con base en esta nueva instrucción intentaré seleccionar de esta forma los best hits. (Sólo con linea y piso).

Referencias:
* https://cmdlinetips.com/2018/02/how-to-subset-pandas-dataframe-based-on-values-of-a-column/

* https://medium.com/@juan.7655/cleansing-messy-excel-files-with-pandas-95a2ba3d8b2d

* https://medium.com/dunder-data/pandas-challenge-2-explain-the-1-000x-speed-difference-when-taking-the-mean-35d72fe10c1

In [1]:
#### Primero algunas variables importantes de ubicación de los archivos y librerías ####

import os
import pandas as pd
import re
import math

#extractpath = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este es para mi no tan nueva PC-Desktop
#csv_path = "Z:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\" #Path para mi PC Desktop

#extractpath = "F:\\DATA_TFM\\TXTs" #Este path es para mi PC en el trabajo
#csv_path = "F:\\DATA_TFM\\CSVs\\" #Path para mi PC en el trabajo

extractpath = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\TXTs" #Este path es para mi laptop en casa
csv_path = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\\" #Path para mi latptop en casa

txtfilesarr = os.listdir(extractpath); txtfilesarr = sorted(txtfilesarr)
csvarr = os.listdir(csv_path); csvarr = sorted(csvarr)

In [13]:
###############################
#### FUNCIONES IMPORTANTES ####
###############################

# Funcion para obtener los indices de un archivo
def indices(df):
    ind_startev = [];ind_runid = [];ind_weights = [];ind_muon = []
    ind_bbfit = [];ind_bb_selp = [];ind_endev = [];#ind_aafit = []
    for i in range(data.shape[0]):
        tempstr = data[0][i]
        if "start_event" in tempstr: ind_startev.append(i)
        if "UTC" in tempstr: ind_runid.append(i)
        if "weights" in tempstr: ind_weights.append(i)
        if "muon" in tempstr: ind_muon.append(i)
        #if "aafit" in tempstr:  ##NO LO CALCULARE!!! OBTENDRE LOS VALORES DE OTRA FORMA
        #    ind_aafit.append(i) ##COMO TENGO EL INDICE DE BBFIT, RESTO 1 LINEA Y ASIGNO VALORES
        if "bbfit" in tempstr:   ##SEGUN LA EXISTENCIA O NO DE LA LINEA!!!! EUREKA!!!
            ind_bbfit.append(i)
        if "selected" in tempstr: ind_bb_selp.append(i)
        if "end_event" in tempstr: ind_endev.append(i)
    return(ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev)

#Función para limpiar un hit (lista) de elementos tipo dobles espacios y simples
def limpiahit(templist):
    cleanlist = []
    for j in range(len(templist)):
        if templist[j]!="":
            cleanlist.append(templist[j])
    return(cleanlist)

#Función para buscar los "rows" que coinciden en línea y piso de solo un selected pulse
def func_lineas_coinc(linea,piso,selp,hits):
    lineas_coinc = []
    for i in range(len(hits)):
        if hits[i][2]==linea and hits[i][3]==piso:
            lineas_coinc.append(hits_raw_limpios[i])
    return(lineas_coinc)

#Función para obtener todas las líneas de los selected pulse de un evento
def func_lineas_evento(sel_hits_limpios,hits_raw_limpios):
    lineas_selp_evento = []
    for i in range(len(sel_hits_limpios)):
        lindet = sel_hits_limpios[i][2] #Linea detector a buscar
        storey = sel_hits_limpios[i][3] #Piso detector a buscar
        lineas = func_lineas_coinc(lindet,storey,sel_hits_limpios[i],hits_raw_limpios)
        #Debo añadir las lineas de cada selected pulse a otra lista como "renglones"
        for j in range(len(lineas)):
            lineas_selp_evento.append(lineas[j])
    return(lineas_selp_evento)

#Función para verificar si los indices tienen el mismo # de elementos
def longitudes(i1,i2,i3,i4,i5,i6,i7):
    a=len(i1);b=len(i2);c=len(i3);d=len(i4);e=len(i5);f=len(i6);g=len(i7)
    #print(a,b,c,d,e,f,g)
    if (a==b and b==c and c==d and d==e and e==f and f==g): return(True)
    else: return(False)
    return()

#Función que hice para buscar un frameID en específico en el dataframe final
def func_frameID(num,df):
    tempdf = df.loc[df['frameID'].isin([str(num)])]
    #print(tempdf.head(5))
    return(tempdf)

## Como ya he visto que trabajar con listas es muchísimo más rapido intentaré hacerlo sólo de esta manera (con listas), al final sí generaré un dataframe, pero sólo hasta el final.
    
## Primero debemos tomar el primer selected pulse como pivote y después debemos comparar contra todas las líneas que coincidan en linea detector y piso detector y seleccionarlas.

In [ ]:
###################################
### OBTENIENDO LOS MEJORES HITS ###
###################################

###### CÓDIGO PARA RECORRER TODOS LOS ARCHIVOS ######

#### FILTRO PARA BUSCAR LOS HITS QUE HACEN MATCH CON EL SELECTED PULSE Y QUEDARNOS 
#### CON LA DATA DE MODULO OPTICO (ENTIENDASE RENGLON O RENGLONES QUE CORRESPONDEN AL SELECTED HIT)
#### EL CRITERIO ES QUEDARSE CON TODOS LOS RAW HITS QUE COINCIDAN EN PISO Y LINEA DETECTOR

#inicio = 432   #En este caso como trabaje los archivos con otra PC, debo empezar en este índice.
best_hits = [] #Definimos la lista donde se guardarán los hits seleccionados

#Primer bucle donde se leerá la data de cada archivo
for z in range(len(csvarr)): #Aqui le deje el len porque estoy trabajando en casa con la laptop
    #Abrimos el archivo, de acuerdo con el índice z
    file = csv_path+csvarr[z]
    data = pd.read_csv(file, header=None)
    #Obteniendo los indices necesarios
    ind_startev,ind_runid,ind_weights,ind_muon,ind_bbfit,ind_bb_selp,ind_endev = indices(data)
    print("El archivo trabajando es:",file,"eventos:",len(ind_startev))
    #Obtenemos la metadata runID e interactionID ya que es la misma para cada archivo
    runID = int(list(str(data[0][ind_runid[0]]).split(" "))[0])#Corregí esta línea era 0 en ambos
    interactionID = [lambda:1, lambda:2]["anumu" in str(file)]()
    print("Entrando al bucle para obtener los best-hits")
    bhcount_file = 0; rhcount_file = 0 #Contadores para "monitorear"
    ### Bucle para buscar los best hits de cada archivo en cada evento del archivo
    for j in range(len(ind_startev)):
        #Obtengo el frameID y trigger_counter, que son diferentes para cada evento
        temprunid = list(data[0][ind_runid[j]].split(" "))
        frameID = temprunid[1]; trigger_counter = temprunid [2]
        #Encontramos primero el # de lineas de rawhits y selected pulses
        difrawhits = ind_bb_selp[j] - ind_bbfit[j] #debe variar con la variable del bucle de eventos
        difselhits = ind_endev[j] - ind_bb_selp[j] #debe variar con la variable del bucle de eventos
        start_rawhit = ind_bbfit[j]+1; #debe variar con el bucle de eventos
        start_selhit = ind_bb_selp[j]+1 #debe variar con el bucle de eventos
        raw_hits = []; hits_raw_limpios = [];sel_hits = []; sel_hits_limpios = []
        #Obteniendo solo los raw_hits de un evento
        for i in range(difrawhits-1):
            temprawhit = list(data[0][start_rawhit+i].split(" "))
            raw_hits.append(temprawhit)
            #Limpiamos los raw hits obtenidos del evento
        for i in range(len(raw_hits)): hits_raw_limpios.append(limpiahit(raw_hits[i]))
        #Obteniendo solo los selected pulses (hits) del mismo evento
        for i in range(difselhits-1):
            tempselhit = list(data[0][start_selhit+i].split(" "))
            sel_hits.append(tempselhit)
        #Limpiamos los raw hits obtenidos del evento
        for i in range(len(sel_hits)): sel_hits_limpios.append(limpiahit(sel_hits[i]))
        #A continuación obtenemos todas las lineas que coinciden con cada selected pulse del evento
        templist = func_lineas_evento(sel_hits_limpios,hits_raw_limpios)
        #Agregamos la metadata que le falta a cada "row"
        for i in range(len(templist)):
            templist[i].insert(0,interactionID)
            templist[i].insert(0,trigger_counter)
            templist[i].insert(0,frameID)
            templist[i].insert(0,runID)
        for k in range(len(templist)):
            best_hits.append(templist[k])
        rhcount_file += len(raw_hits)
        bhcount_file += len(templist)
    print("Raw_hits:",rhcount_file,"Número de best_hits:",bhcount_file)
    #inicio += 1 #Incremento para abrir el archivo correcto

print("\nFin de la extracción de la data")

El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025800_anumu_CC_a_reco.i3.csv eventos: 256
Entrando al bucle para obtener los best-hits
Raw_hits: 41006 Número de best_hits: 10018
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025800_numu_CC_a_reco.i3.csv eventos: 223
Entrando al bucle para obtener los best-hits
Raw_hits: 35508 Número de best_hits: 8469
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025880_anumu_CC_a_reco.i3.csv eventos: 271
Entrando al bucle para obtener los best-hits
Raw_hits: 43087 Número de best_hits: 10554
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025880_numu_CC_a_reco.i3.csv eventos: 230
Entrando al bucle para obtener los best-hits
Raw_hits: 35388 Número de best_hits: 7889
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_025920_anumu_CC_a_reco.i3.csv eventos: 273
Entrando al bucle para obtener los best-hits
Raw_hits: 37399 Número de best_hits: 10225
El a

Raw_hits: 22866 Número de best_hits: 5066
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026800_anumu_CC_a_reco.i3.csv eventos: 219
Entrando al bucle para obtener los best-hits
Raw_hits: 49446 Número de best_hits: 10666
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026800_numu_CC_a_reco.i3.csv eventos: 180
Entrando al bucle para obtener los best-hits
Raw_hits: 40495 Número de best_hits: 8531
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026810_anumu_CC_a_reco.i3.csv eventos: 210
Entrando al bucle para obtener los best-hits
Raw_hits: 46115 Número de best_hits: 8551
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026810_numu_CC_a_reco.i3.csv eventos: 178
Entrando al bucle para obtener los best-hits
Raw_hits: 40678 Número de best_hits: 8686
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_026820_anumu_CC_a_reco.i3.csv eventos: 250
Entrando al bucle para obtener los best-hits
Raw_hi

El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027550_numu_CC_a_reco.i3.csv eventos: 260
Entrando al bucle para obtener los best-hits
Raw_hits: 42220 Número de best_hits: 9957
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027580_anumu_CC_a_reco.i3.csv eventos: 218
Entrando al bucle para obtener los best-hits
Raw_hits: 32668 Número de best_hits: 8470
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027580_numu_CC_a_reco.i3.csv eventos: 242
Entrando al bucle para obtener los best-hits
Raw_hits: 36090 Número de best_hits: 9296
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027620_anumu_CC_a_reco.i3.csv eventos: 158
Entrando al bucle para obtener los best-hits
Raw_hits: 19514 Número de best_hits: 5751
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_027620_numu_CC_a_reco.i3.csv eventos: 156
Entrando al bucle para obtener los best-hits
Raw_hits: 19219 Número de best_hits: 5587
El archi

Raw_hits: 48701 Número de best_hits: 8467
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_028570_numu_CC_a_reco.i3.csv eventos: 148
Entrando al bucle para obtener los best-hits
Raw_hits: 37058 Número de best_hits: 6937
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_028620_anumu_CC_a_reco.i3.csv eventos: 238
Entrando al bucle para obtener los best-hits
Raw_hits: 59485 Número de best_hits: 10041
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_028620_numu_CC_a_reco.i3.csv eventos: 206
Entrando al bucle para obtener los best-hits
Raw_hits: 52001 Número de best_hits: 9525
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_028630_anumu_CC_a_reco.i3.csv eventos: 317
Entrando al bucle para obtener los best-hits
Raw_hits: 57745 Número de best_hits: 11575
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_028630_numu_CC_a_reco.i3.csv eventos: 286
Entrando al bucle para obtener los best-hits
Raw_hi

El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029120_anumu_CC_a_reco.i3.csv eventos: 455
Entrando al bucle para obtener los best-hits
Raw_hits: 58800 Número de best_hits: 13025
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029120_numu_CC_a_reco.i3.csv eventos: 430
Entrando al bucle para obtener los best-hits
Raw_hits: 56537 Número de best_hits: 13278
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029160_anumu_CC_a_reco.i3.csv eventos: 387
Entrando al bucle para obtener los best-hits
Raw_hits: 58482 Número de best_hits: 12702
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029160_numu_CC_a_reco.i3.csv eventos: 334
Entrando al bucle para obtener los best-hits
Raw_hits: 50256 Número de best_hits: 11097
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_029180_anumu_CC_a_reco.i3.csv eventos: 368
Entrando al bucle para obtener los best-hits
Raw_hits: 50274 Número de best_hits: 11116
El

...

Raw_hits: 201253 Número de best_hits: 30411
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_032740_numu_CC_a_reco.i3.csv eventos: 1017
Entrando al bucle para obtener los best-hits
Raw_hits: 180529 Número de best_hits: 27153
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_032760_anumu_CC_a_reco.i3.csv eventos: 1110
Entrando al bucle para obtener los best-hits
Raw_hits: 201310 Número de best_hits: 29646
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_032760_numu_CC_a_reco.i3.csv eventos: 1010
Entrando al bucle para obtener los best-hits
Raw_hits: 183093 Número de best_hits: 28271
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_032770_anumu_CC_a_reco.i3.csv eventos: 1090
Entrando al bucle para obtener los best-hits
Raw_hits: 198790 Número de best_hits: 28368
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_032770_numu_CC_a_reco.i3.csv eventos: 958
Entrando al bucle para obtener los best

Raw_hits: 183869 Número de best_hits: 21694
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_033530_anumu_CC_a_reco.i3.csv eventos: 747
Entrando al bucle para obtener los best-hits
Raw_hits: 182887 Número de best_hits: 23221
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_033530_numu_CC_a_reco.i3.csv eventos: 704
Entrando al bucle para obtener los best-hits
Raw_hits: 171366 Número de best_hits: 21009
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_033570_anumu_CC_a_reco.i3.csv eventos: 703
Entrando al bucle para obtener los best-hits
Raw_hits: 172020 Número de best_hits: 20308
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_033570_numu_CC_a_reco.i3.csv eventos: 693
Entrando al bucle para obtener los best-hits
Raw_hits: 168026 Número de best_hits: 19707
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_033590_anumu_CC_a_reco.i3.csv eventos: 990
Entrando al bucle para obtener los best-hi

Raw_hits: 219257 Número de best_hits: 26257
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_035000_anumu_CC_a_reco.i3.csv eventos: 964
Entrando al bucle para obtener los best-hits
Raw_hits: 251209 Número de best_hits: 29837
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_035000_numu_CC_a_reco.i3.csv eventos: 880
Entrando al bucle para obtener los best-hits
Raw_hits: 225897 Número de best_hits: 25891
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_035180_anumu_CC_a_reco.i3.csv eventos: 1441
Entrando al bucle para obtener los best-hits
Raw_hits: 434500 Número de best_hits: 46252
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_035180_numu_CC_a_reco.i3.csv eventos: 1343
Entrando al bucle para obtener los best-hits
Raw_hits: 405762 Número de best_hits: 44924
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_035190_anumu_CC_a_reco.i3.csv eventos: 1411
Entrando al bucle para obtener los best

Raw_hits: 386364 Número de best_hits: 40656
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_036940_anumu_CC_a_reco.i3.csv eventos: 1476
Entrando al bucle para obtener los best-hits
Raw_hits: 405727 Número de best_hits: 43405
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_036950_anumu_CC_a_reco.i3.csv eventos: 1491
Entrando al bucle para obtener los best-hits
Raw_hits: 401433 Número de best_hits: 43554
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_037230_numu_CC_a_reco.i3.csv eventos: 1306
Entrando al bucle para obtener los best-hits
Raw_hits: 367641 Número de best_hits: 39631
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_037280_anumu_CC_a_reco.i3.csv eventos: 1524
Entrando al bucle para obtener los best-hits
Raw_hits: 380688 Número de best_hits: 41101
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_037300_anumu_CC_a_reco.i3.csv eventos: 1526
Entrando al bucle para obtener los b

Raw_hits: 265792 Número de best_hits: 29878
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_038770_numu_CC_a_reco.i3.csv eventos: 970
Entrando al bucle para obtener los best-hits
Raw_hits: 266840 Número de best_hits: 28985
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_038780_anumu_CC_a_reco.i3.csv eventos: 1179
Entrando al bucle para obtener los best-hits
Raw_hits: 316955 Número de best_hits: 33864
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_038780_numu_CC_a_reco.i3.csv eventos: 987
Entrando al bucle para obtener los best-hits
Raw_hits: 269677 Número de best_hits: 31573
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_038790_anumu_CC_a_reco.i3.csv eventos: 1041
Entrando al bucle para obtener los best-hits
Raw_hits: 281244 Número de best_hits: 29780
El archivo trabajando es: U:\MASTER INT. ARTIFICIAL\TFM_DATA\CSVs\MC_038790_numu_CC_a_reco.i3.csv eventos: 957
Entrando al bucle para obtener los best-h

In [ ]:
#Pasamos a dataframe la lista con todos los hits seleccionados, en este caso de un archivo
print(len(best_hits))
columnas = ['runID','frameID','trigger_counter','interactionID','hit','#hit','lin_det','piso_det','mód_opt',
            'X\'','Y\'','Z\'','X','Y','Z','tiempo','amp','frec_hits']
df = pd.DataFrame(best_hits,columns=columnas)

In [ ]:
df.sample(5)

In [ ]:
#Guardamos el dataframe como un archivo CSV para unirlo con el otro ya procesado
#Todavía falta generar la transformación azimut y zenit pero eso será cuando ya tenga el dataframe completo

outputpath = "U:\MASTER INT. ARTIFICIAL\TFM_DATA\CLEANDATA\\"
#outputpath = "F:\DATA_TFM\CLEANDATA\\"
    
df.to_csv(outputpath+"data_best_hits.csv", index = None)#, header=None)